In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor, AutoModelForPreTraining, Trainer, TrainingArguments
from datasets import Dataset
from datasets import Dataset as HFDataset
from datasets import load_dataset
import evaluate
import torch
from torch.utils.data import Dataset, DataLoader
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

from huggingface_hub import login
login()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
olist_df = pd.read_csv(r"/content/drive/MyDrive/E-Commerce-Chatbot/Data/df_sampled_olist.csv")

olist_df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,...,geolocation_city,geolocation_state,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,product_category,comparator_price
0,5eb5db738b8f5ebe2a07fe4fd5af2b8c,8fbefa23ae0c37b2b4ca94dd96abdb78,21230,rio de janeiro,RJ,26a44b28ac942b09a771479c880882d9,delivered,2017-07-30 19:23:33,2017-07-30 19:35:18,2017-08-02 19:24:00,...,rio de janeiro,RJ,ceb5e7e8598d5c949e346aca45a4c8e0,5.0,NaN,Ótimo,2017-08-10 00:00:00,2017-08-10 23:42:46,Home & Garden,47.278362
1,2bd62951b26a453e99cea4aaf432ed45,39518dab7aa8a79da80322d53bf504d4,4005,sao paulo,SP,7a93502a455e1399090b442409cf516c,delivered,2018-07-27 14:58:55,2018-07-27 15:31:16,2018-08-01 18:13:00,...,sao paulo,SP,be9d505f73a72ca9af17ed02926d8d7b,3.0,NaN,NaN,2018-08-07 00:00:00,2018-08-07 22:25:09,Furniture,90.199163
2,551adb1ab4dd9254a79d487f163c9629,305af21f2178486da285dd0082ae5240,8371,sao paulo,SP,62899f7b8e4122204fc3d26d8d2c3737,delivered,2017-07-19 11:34:45,2017-07-20 11:43:36,2017-07-20 17:54:52,...,sao paulo,SP,8848a9dafb4a3f6eea027ee2322ff6d4,5.0,NaN,NaN,2017-08-05 00:00:00,2017-08-08 13:47:58,Home & Garden,110.554609
3,b71cd39228d8554a93375e867f35e724,12613f1b2693e38c011a5c20a9e2f2a8,38610,unai,MG,9de65156d923c62015d982bd3f6ade9b,delivered,2018-02-28 09:42:13,2018-03-03 02:49:38,2018-03-08 18:19:45,...,unai,MG,60b32cb1df5164832f4a283b32e6dcc5,1.0,NaN,Realizei a compra de um Toner Original CE320A ...,2018-03-23 00:00:00,2018-03-28 14:41:51,Electronics,212.687803
4,d4ba90ce0c49bffcd1b221f4bd2e9741,08a151ec3a5cbe140edd1fa285f6c853,25550,sao joao de meriti,RJ,284f45ebbf2b9dae5ad6699417798e08,delivered,2017-07-01 15:09:06,2017-07-01 15:24:00,2017-07-06 15:45:57,...,sao joao de meriti,RJ,78b6a2a48db20e7d70c5ad2ef90c3024,5.0,NaN,NaN,2017-07-19 00:00:00,2017-07-20 02:04:54,Home & Garden,36.167473


In [9]:
olist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 47 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   customer_id                    20000 non-null  object 
 1   customer_unique_id             20000 non-null  object 
 2   customer_zip_code_prefix       20000 non-null  int64  
 3   customer_city                  20000 non-null  object 
 4   customer_state                 20000 non-null  object 
 5   order_id                       20000 non-null  object 
 6   order_status                   20000 non-null  object 
 7   order_purchase_timestamp       20000 non-null  object 
 8   order_approved_at              19997 non-null  object 
 9   order_delivered_carrier_date   19794 non-null  object 
 10  order_delivered_customer_date  19616 non-null  object 
 11  order_estimated_delivery_date  20000 non-null  object 
 12  payment_sequential             20000 non-null 

In [10]:
prompt_df = pd.read_csv(
    "hf://datasets/bitext/Bitext-customer-support-llm-chatbot-training-dataset/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv"
)

prompt_df.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [11]:
# Prompt Dataset Information and Details
prompt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26872 entries, 0 to 26871
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   flags        26872 non-null  object
 1   instruction  26872 non-null  object
 2   category     26872 non-null  object
 3   intent       26872 non-null  object
 4   response     26872 non-null  object
dtypes: object(5)
memory usage: 1.0+ MB


In [12]:
pattern = r'\{\{(.*?)\}\}'

special = prompt_df['instruction'].apply(lambda x: re.findall(pattern, x)[0] if re.findall(pattern, x) else None)

special.value_counts()

,count
instruction,
Order Number,2907
Account Type,1011
Person Name,887
Account Category,822
Currency Symbol,372
Refund Amount,252
Delivery City,234
Delivery Country,177
Invoice Number,8


In [13]:
replacement_map = {
    "Order Number": "order_id",
    "Account Type": "user",
    "Person Name": "customer_unique_id",
    "Account Category": "account_category",
    "Currency Symbol": "$",
    "Refund Amount": "freight_value",
    "Delivery City": "geolocation_city",
    "Delivery Country": "geolocation_state",
    "Invoice Number": "order_item_id"
}


def replace_special_tokens(instruction, data):
    for token, column in replacement_map.items():
        # Check if the token is present in the instruction
        if f"{{{token}}}" in instruction:
            # Get a random value from the column if it exists
            if column in data.columns:
                value = data[column].sample(1, random_state = 42).values[0]
            else:
                # Handle cases where the column doesn't exist
                value = f"{column}"
            instruction = instruction.replace(f"{{{token}}}", str(value))

    return instruction


# instruction =  "What is the status of the order? {{Order Number}} "
# replace_special_tokens(instruction, olist_df)

prompt_df['instruction'] = prompt_df.apply(lambda row: replace_special_tokens(row['instruction'], olist_df), axis=1)
prompt_df.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {9cb82a3ae51fb...,ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {9cb...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {9cb82a3ae51fb0...,ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {9cb82a3ae51fb07b957...,ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {9...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [14]:
new_rows = [
    {"instruction": "what is the store website?", "response": "The store website is 'www.oliststore.com'", "intent": "get_store_info", "category": "general_info", "flags": "website"},
    {"instruction": "what's the store's website address?", "response": "The store website is 'www.oliststore.com'", "intent": "get_store_info", "category": "general_info", "flags": "website"},
    {"instruction": "can you tell me the store website?", "response": "The store website is 'www.oliststore.com'", "intent": "get_store_info", "category": "general_info", "flags": "website"},
    {"instruction": "where can I find the store online?", "response": "The store website is 'www.oliststore.com'", "intent": "get_store_info", "category": "general_info", "flags": "website"},
    {"instruction": "do you have a website?", "response": "The store website is 'www.oliststore.com'", "intent": "get_store_info", "category": "general_info", "flags": "website"}
]

# Concatenate DataFrames using `pd.concat`
prompt_df = pd.concat([prompt_df, pd.DataFrame(new_rows)], ignore_index=True)

prompt_df.tail()

,flags,instruction,category,intent,response
26872,website,what is the store website?,general_info,get_store_info,The store website is 'www.oliststore.com'
26873,website,what's the store's website address?,general_info,get_store_info,The store website is 'www.oliststore.com'
26874,website,can you tell me the store website?,general_info,get_store_info,The store website is 'www.oliststore.com'
26875,website,where can I find the store online?,general_info,get_store_info,The store website is 'www.oliststore.com'
26876,website,do you have a website?,general_info,get_store_info,The store website is 'www.oliststore.com'


In [17]:
class DialogueDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # Combine instruction and response with special tokens
        dialogue = f"{row['instruction']} [SEP] {row['response']}"

        # Tokenize the text
        encodings = self.tokenizer(
            dialogue,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )

        # Create labels (same as input_ids for language modeling)
        labels = encodings.input_ids.clone()

        # Convert to tensor format and squeeze extra dimension
        return {
            'input_ids': encodings.input_ids.squeeze(),
            'attention_mask': encodings.attention_mask.squeeze(),
            'labels': labels.squeeze()
        }

def prepare_data(df, tokenizer, train_size=0.9, max_length=128):
    # Convert DataFrame to Hugging Face Dataset
    dataset_dict = {
        'instruction': df['instruction'].tolist(),
        'response': df['response'].tolist()
    }
    dataset = HFDataset.from_dict(dataset_dict)

    # Split dataset
    dataset = dataset.train_test_split(train_size=train_size)

    def tokenize_function(examples):
        # Combine instruction and response
        dialogues = [f"{instr} {tokenizer.eos_token} {resp}"
                    for instr, resp in zip(examples['instruction'], examples['response'])]

        # Tokenize
        tokenized = tokenizer(
            dialogues,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors='pt'
        )

        # Create labels (shift input_ids right by one position)
        labels = tokenized['input_ids'].clone()
        labels[labels == tokenizer.pad_token_id] = -100  # Ignore padding tokens in loss

        return {
            'input_ids': tokenized['input_ids'],
            'attention_mask': tokenized['attention_mask'],
            'labels': labels
        }

    # Tokenize datasets
    tokenized_dataset = dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=dataset['train'].column_names
    )

    return tokenized_dataset


In [18]:
def train_model(df, model_name="microsoft/DialoGPT-small", output_dir="./fine_tuned_dialo-gpt"):
    # Initialize tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Ensure padding token is set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id

    # Prepare dataset
    tokenized_dataset = prepare_data(df, tokenizer)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=100,
        eval_strategy="steps",
        eval_steps=500,
        save_strategy="steps",
        save_steps=1000,
        load_best_model_at_end=True,
        remove_unused_columns=False
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset['train'],
        eval_dataset=tokenized_dataset['test'],
    )

    # Train the model
    trainer.train()

    return model, tokenizer

In [19]:
# Train the modelb
model, tokenizer = train_model(prompt_df,
                               model_name = "openai-community/gpt2-medium",
                               output_dir = "./fine_tuned_gpt-2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,1.097800,1.008596
1000,0.918800,0.868525
1500,0.851700,0.803866
2000,0.813500,0.767539
2500,0.777900,0.743315
3000,0.754000,0.725036
3500,0.707100,0.712517
4000,0.683400,0.703320
4500,0.689800,0.693644
5000,0.660100,0.686998


Step,Training Loss,Validation Loss
500,1.097800,1.008596
1000,0.918800,0.868525
1500,0.851700,0.803866
2000,0.813500,0.767539
2500,0.777900,0.743315
3000,0.754000,0.725036
3500,0.707100,0.712517
4000,0.683400,0.703320
4500,0.689800,0.693644
5000,0.660100,0.686998


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


In [20]:
# Push model and tokenizer directly to Hugging Face Hub
model.push_to_hub("khalednabawi11/fine_tuned_gpt-2")
tokenizer.push_to_hub("khalednabawi11/fine_tuned_gpt-2")

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/khalednabawi11/fine_tuned_gpt-2/commit/8a5aebf731f86d0c6c48574fe8c20e0f8a907c60', commit_message='Upload tokenizer', commit_description='', oid='8a5aebf731f86d0c6c48574fe8c20e0f8a907c60', pr_url=None, pr_revision=None, pr_num=None)

In [25]:
def clean_response(response):
    # Remove placeholders that look like {{Token}} using regular expressions
    cleaned_response = re.sub(r"\{\{.*?\}\}", "", response).strip()
    cleaned_response = re.sub(r'\s+', ' ', response).strip()
    return cleaned_response


def generate_response(prompt, max_length=150):
    # Set device (GPU if available, else CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Define tokens to remove from the prompt
    tokens_to_remove = [
        '{{Order Number}}', '{{Website URL}}', '{{Account Type}}', '{{Person Name}}',
        '{{Account Category}}', '{{Currency Symbol}}', '{{Refund Amount}}',
        '{{Delivery City}}', '{{Delivery Country}}', '{{Invoice Number}}'
    ]

    # Clean up the prompt by removing unnecessary placeholders
    clean_prompt = prompt
    for token in tokens_to_remove:
        clean_prompt = clean_prompt.replace(token, "").strip()

    # Tokenize input and move inputs to the correct device
    inputs = tokenizer(
        clean_prompt,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to(device)

    # Generate outputs with specified parameters for better control over generation
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        # Adjust based on input length
        max_length=max_length + inputs["input_ids"].shape[1],
        do_sample=True,
        temperature=0.6,
        no_repeat_ngram_size=3,
        num_beams=3,
        early_stopping=True,
        top_p=0.92,
        top_k=50,
        repetition_penalty=1.2
    )

    # Decode the response and move back to CPU
    response = tokenizer.decode(outputs[0].to('cpu'), skip_special_tokens=True)

    # Remove the clean prompt from the response
    response = response.replace(clean_prompt, '').strip()

    # Remove any unwanted tokens from the response
    for token in tokens_to_remove:
        response = response.replace(token, "").strip()

    # Split response by new lines and strip spaces
    lines = [line.strip()
             for line in re.split(r'\n', response) if line.strip()]

    # Clean up periods before digits if needed
    lines = [re.sub(r'\s*\.\s*(?=\d)', "", line) for line in lines]

    # Join the lines into a single response, formatting numbered steps properly
    response = '\n'.join(lines).strip()

    # Ensure the response ends on a complete sentence
    if response:
        last_punct_index = max(response.rfind(
            '.'), response.rfind('!'), response.rfind('?'))
        if last_punct_index != -1:
            response = response[:last_punct_index + 1].strip()

    # Return the final cleaned response
    return response

# Test the improved generation
prompt = "How can i pay for product {9cb82a3ae51fb}?"
response = generate_response(prompt)
print("Prompt:", prompt)
print("\nResponse:", response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: How can i pay for product {9cb82a3ae51fb}?

Response: To pay for your product, you can follow these steps:
1. Visit our website and log in to your account.
2. Navigate to the "My Orders" or "Order History" section.
3. Look for the specific order with the order number .
4. Click on the order to view the details of the product you want to purchase.
5. On the product page, you will find an option to "Payment Methods." Select this option to proceed.
6. Follow the on-screen instructions to complete the payment.
If you encounter any difficulties or have further questions, feel free to reach out to our customer support team.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the fine-tuned model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("khalednabawi11/fine_tuned_dialo-gpt")
model = AutoModelForCausalLM.from_pretrained("khalednabawi11/fine_tuned_dialo-gpt")

# Function to generate a response
def generate_response(user_input, conversation_history=None):
    if conversation_history is None:
        conversation_history = ""

    # Combine conversation history and new user input
    input_text = conversation_history + f"User: {user_input}\nBot:"

    # Tokenize the input
    inputs = tokenizer.encode(input_text, return_tensors="pt")

    # Generate response
    outputs = model.generate(inputs, max_length=150, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the bot's reply from the response
    bot_reply = response.split("Bot:")[-1].strip()

    return bot_reply, response

# Example usage
user_input = "What is the delivery city for my order?"
bot_response, full_response = generate_response(user_input)

print("Bot Response:", bot_response)

Bot Response: We understand that you would like to know the delivery city for your order. To provide you with accurate information, could you please provide us with the {{Delivery City}} or {{Delivery City}}? With this information, we will be able to assist you further in determining the delivery location. Your satisfaction is our top priority, and we are here to ensure a smooth and seamless delivery experience for you. Please let us know the specific city or region you are looking for, and we will be more than happy to provide you with the delivery details. Your satisfaction is our top priority


In [36]:
prompt_df['input_ids'] = prompt_df['instruction'].apply(lambda x: tokenizer.encode(x, return_tensors='pt').squeeze(0))
prompt_df['target_ids'] = prompt_df['response'].apply(lambda x: tokenizer.encode(x, return_tensors='pt').squeeze(0))

# Sample the dataset
test_dataset = prompt_df.sample(2000)

def evaluate_model(model, tokenizer, test_dataset, batch_size=32):
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    smooth = SmoothingFunction().method1

    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    bleu_scores = []

    model.eval()
    num_examples = len(test_dataset)
    num_batches = (num_examples + batch_size - 1) // batch_size

    with torch.no_grad():
        for i in tqdm(range(num_batches)):
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, num_examples)
            batch = test_dataset[start_idx:end_idx]

            input_texts = []
            reference_texts = []

            # Access the input_ids and target_ids from the batch
            for input_ids, target_ids in zip(batch['input_ids'], batch['target_ids']):
                input_texts.append(tokenizer.decode(input_ids, skip_special_tokens=False))
                reference_texts.append(tokenizer.decode(target_ids, skip_special_tokens=False))

            # Generate predictions
            inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)
            outputs = model.generate(
                inputs["input_ids"].to(model.device),
                max_length=100,
                num_return_sequences=1,
                temperature=0.7,
                no_repeat_ngram_size=2,
                pad_token_id=tokenizer.pad_token_id
            )

            predicted_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

            # Calculate scores for each example in batch
            for pred, ref in zip(predicted_texts, reference_texts):
                # ROUGE scores
                rouge_scores = rouge.score(pred, ref)
                rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
                rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
                rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

                # BLEU score
                pred_tokens = pred.split()
                ref_tokens = ref.split()
                if len(pred_tokens) > 0 and len(ref_tokens) > 0:
                    bleu = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smooth)
                    bleu_scores.append(bleu)

    results = {
        'rouge1': np.mean(rouge1_scores),
        'rouge2': np.mean(rouge2_scores),
        'rougeL': np.mean(rougeL_scores),
        'bleu': np.mean(bleu_scores)
    }

    results.update({
        'rouge1_std': np.std(rouge1_scores),
        'rouge2_std': np.std(rouge2_scores),
        'rougeL_std': np.std(rougeL_scores),
        'bleu_std': np.std(bleu_scores)
    })

    return results

In [37]:
def print_evaluation_results(results):
    """Pretty print the evaluation results"""
    print("\nEvaluation Results:")
    print("=" * 50)
    metrics = [
        ('ROUGE-1', 'rouge1'),
        ('ROUGE-2', 'rouge2'),
        ('ROUGE-L', 'rougeL'),
        ('BLEU', 'bleu')
    ]

    for metric_name, metric_key in metrics:
        mean = results[metric_key]
        std = results[f'{metric_key}_std']
        print(f"{metric_name:8} : {mean:.4f} ± {std:.4f}")

results = evaluate_model(model, tokenizer, test_dataset)
print_evaluation_results(results)

  0%|          | 0/63 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
100%|██████████| 63/63 [02:36<00:00,  2.49s/it]


Evaluation Results:
ROUGE-1  : 0.4994 ± 0.1097
ROUGE-2  : 0.2654 ± 0.1064
ROUGE-L  : 0.3426 ± 0.1078
BLEU     : 0.1387 ± 0.1060
